In [1]:
!pip3 install -q -U git+https://github.com/huggingface/transformers.git

!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
!pip3 install -q -U git+https://github.com/huggingface/optimum.git

!pip3 install -q -U flash-attn
!pip install -q -U bitsandbytes
!pip install -q -U accelerate


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer
import torch
import re
import json

In [3]:
model_id = "TheBloke/CodeLlama-7B-Instruct-GPTQ"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
          bnb_4bit_use_double_quant=True,
          bnb_4bit_quant_type="nf4",
          bnb_4bit_compute_dtype=torch.bfloat16
 )

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
    torch_dtype=torch.float16,
    revision = "main",
    trust_remote_code = True,
    cache_dir = ''
    )
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

configuration_llama.py:   0%|          | 0.00/8.56k [00:00<?, ?B/s]

modeling_llama.py:   0%|          | 0.00/45.9k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [4]:
# Use prompts created for the zero-shot task
with open("/content/zero_prompts.json", "r") as file:
    zero_prompts = json.load(file)


In [5]:
# Delimiters
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

system_prompt="""You are a helpful assistant. You provide accurate information."""

# Define a stream *without* function calling capabilities
def stream(user_prompt):
    prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}{E_SYS}{user_prompt.strip()} {E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").input_ids.cuda()
    streamer = TextStreamer(tokenizer)

    with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
        generated_text = model.generate(inputs=inputs, streamer=streamer, max_new_tokens=500, temperature=0.01, do_sample=True)

    # Decode the generated text
    generated_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)

    return generated_text




In [10]:
cllama_ans = {}

# Run the stream twice
for i in range(2):
    for question in zero_prompts:
        # Call the stream function to get the answer
        answer = stream(zero_prompts[question])

        # Extract the answer dictionary from the response
        answer_clean = re.findall(r'\{([^{}]*)\}', answer)
        answer_dicts = []
        for item in answer_clean:
            # Split by comma or newline
            items_split = re.split(r',|\n', item)
            # Create dictionary from split items
            pairs = [pair.split(':') for pair in items_split]
            # Filter out invalid pairs
            valid_pairs = [pair for pair in pairs if len(pair) == 2]
            # Create dictionary from valid pairs
            answer_dict = dict(valid_pairs)
            answer_dicts.append(answer_dict)
        # Add the answer dictionary to the list of answers for the current question
        if question not in cllama_ans:
            cllama_ans[question] = []
        cllama_ans[question].extend(answer_dicts)

        torch.cuda.empty_cache()

<s> [INST] <<SYS>>
You are a helpful assistant. You provide accurate information.
<</SYS>>

In the following you will be given a question followed several possible answer categories. Please     estimate how a representative sample of 7900 US-Americans would have answered the question in 2020 by providing the    number of respondents for each answer option in percent (output a dictionary object with answer category number as    key and answer frequency in percent as value. Only provide the number without the percentage sign).    Here is the questions: There has been some discussion about abortion during recent
years. Which one of the opinions on this page best agrees with
your view?
1. By law, abortion should never be permitted
2. The law should permit abortion only in case of rape, incest, or
when the woman's life is in danger
3. The law should permit abortion other than for
rape/incest/danger to woman but only after need clearly
established
4. By law, a woman should always be able to 

In [21]:
cllama_ans

cllama_answers = json.dumps(cllama_ans)
with open("cllama_answers.json", "w") as file:
    file.write(cllama_answers)

In [22]:
cllama_ans

{'V201336': [{'1': ' 10.3%', '2': ' 21.5%', '3': ' 31.8%', '4': ' 27.4%'},
  {'1': ' 10.5%', ' 2': ' 21.5%', ' 3': ' 47.5%', ' 4': ' 21.5%'}],
 'V201342x': [{'1': ' 30.5%',
   '2': ' 21.5%',
   '3': ' 12.5%',
   '4': ' 26.5%',
   '5': ' 10.5%',
   '6': ' 5.5%',
   '7': ' 0.5%'},
  {'1': ' 30.5%',
   '2': ' 21.5%',
   '3': ' 12.5%',
   '4': ' 25.5%',
   '5': ' 10.5%',
   '6': ' 5.5%',
   '7': ' 0.5%'}],
 'V202332': [{'1': ' 10.5%',
   '2': ' 22.5%',
   '3': ' 32.5%',
   '4': ' 15.5%',
   '5': ' 10.5%'},
  {'1': ' 10.5%', '2': ' 22.5%', '3': ' 32.5%', '4': ' 15.5%', '5': ' 10.5%'}],
 'V202333': [{'1': ' 20.5%',
   '2': ' 22.5%',
   '3': ' 31.5%',
   '4': ' 15.5%',
   '5': ' 10.5%'},
  {'1': ' 20.5%', '2': ' 22.5%', '3': ' 31.5%', '4': ' 15.5%', '5': ' 10.5%'}],
 'V202337': [{'1': ' 30.5%', '2': ' 27.5%', '3': ' 31.5%'},
  {'1': ' 30.5%', '2': ' 27.5%', '3': ' 31.5%'}],
 'V202341x': [{1: ' 10.5%',
   2: ' 22.5%',
   3: ' 25.5%',
   4: ' 42.5%',
   5: ' 15.5%',
   6: ' 17.5%',
   7: ' 10.5